In [8]:
# ==============================================================================
# Cell 1: Imports & Environment Setup
# ==============================================================================
import lyricsgenius
import pandas as pd
import os
import time
import json
from dotenv import load_dotenv

# 1. Load Environment Variables
# This looks for a .env file in the root directory
# load_dotenv() returns True if it finds the file
if not load_dotenv(dotenv_path='../.env'):
    print("⚠️ WARNING: .env file not found in root directory.")
else:
    print("✅ Environment variables loaded.")

# 2. Define File Paths
# We use relative paths so this works on any machine
RAW_DATA_PATH = '../data/raw'
PROCESSED_DATA_PATH = '../data/processed'

# 3. Create Directories (Safety Check)
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print(f"Raw Data Path: {RAW_DATA_PATH}")
print(f"Processed Data Path: {PROCESSED_DATA_PATH}")

✅ Environment variables loaded.
Raw Data Path: ../data/raw
Processed Data Path: ../data/processed


In [9]:
# ==============================================================================
# Cell 2: API Authentication
# ==============================================================================
# Retrieve the token securely from the environment
GENIUS_ACCESS_TOKEN = os.getenv("GENIUS_ACCESS_TOKEN")

if not GENIUS_ACCESS_TOKEN:
    raise ValueError("❌ Error: GENIUS_ACCESS_TOKEN not found in .env file!")

print("Attempting to authenticate with Genius API...")

try:
    # Initialize Genius Object
    # timeout=15: Gives the API 15 seconds before giving up (good for slow connections)
    # sleep_time=0.5: Pauses half a second between calls to be polite
    # retries=3: Tries 3 times if the API fails before throwing an error
    genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN, 
                                 timeout=15, 
                                 retries=3, 
                                 sleep_time=0.5,
                                 verbose=True)
    
    # Clean up output: Don't print massive blocks of text headers
    genius.remove_section_headers = True 
    
    print("✅ Genius Client Authenticated Successfully.")
    
except Exception as e:
    print(f"❌ Authentication Failed: {e}")

Attempting to authenticate with Genius API...
✅ Genius Client Authenticated Successfully.


In [10]:
# ==============================================================================
# Cell 3: Configuration (Scaled-Up Seed Artists)
# ==============================================================================
# We are expanding to 14 artists to capture distinct sub-genres:
# Lyrical, Trap, Melodic/Pop, and Experimental.

SEED_ARTISTS = [
    # --- The Lyricists / Conscious ---
    "Kendrick Lamar", 
    "J. Cole", 
    "Nas",
    "Tyler, the Creator",
    
    # --- The Trap / South Giants ---
    "Future", 
    "Travis Scott", 
    "Young Thug",
    "Lil Baby",
    "Migos",
    
    # --- The Melodic / Pop Bridges ---
    "Drake", 
    "The Weeknd",
    "Post Malone",
    
    # --- The Legends / Producers ---
    "Kanye West",
    "Eminem"
]

# Parameters
# We increase this to 50 to get deep cuts, not just radio hits.
# This ensures we capture more obscure featured artists.
SONGS_PER_ARTIST = 50 

print(f"🚀 SCALING UP: Targeting {len(SEED_ARTISTS)} seed artists.")
print(f"🎯 DEPTH: Collecting top {SONGS_PER_ARTIST} songs per artist.")
print(f"📊 EST. TOTAL: ~{len(SEED_ARTISTS) * SONGS_PER_ARTIST} songs.")

🚀 SCALING UP: Targeting 14 seed artists.
🎯 DEPTH: Collecting top 50 songs per artist.
📊 EST. TOTAL: ~700 songs.


In [11]:
# ==============================================================================
# Cell 4: The Crawl Function
# ==============================================================================
def crawl_artist_data(artist_name, max_songs):
    """
    Searches for an artist, downloads their top songs/lyrics, 
    and saves the raw object to a JSON file.
    """
    print(f"\n--- Starting crawl for: {artist_name} ---")
    
    # Create a clean filename (e.g., "kendrick_lamar_data.json")
    clean_name = artist_name.replace(" ", "_").lower()
    file_path = os.path.join(RAW_DATA_PATH, f"{clean_name}_data.json")
    
    # CACHING STRATEGY:
    # If we already downloaded this artist, skip them to save time/API calls.
    if os.path.exists(file_path):
        print(f"⚡ Cache hit: Data for '{artist_name}' already exists.")
        print(f"   Loading from {file_path}...")
        with open(file_path, 'r') as f:
            # We load it just to return the object, but we don't re-download
            return json.load(f)
    
    # If not in cache, download from Genius
    try:
        # 'sort="popularity"' ensures we get the most important songs first
        artist = genius.search_artist(artist_name, max_songs=max_songs, sort="popularity")
        
        if artist:
            # Save the raw data immediately
            # This saves EVERYTHING (lyrics, metadata, features) in one file
            artist.save_lyrics(filename=file_path, overwrite=True, verbose=True)
            print(f"✅ Download complete. Saved to {file_path}")
            return artist
        else:
            print(f"⚠️ Artist '{artist_name}' not found.")
            return None
        
    except Exception as e:
        print(f"❌ Error crawling {artist_name}: {e}")
        return None

In [12]:
# ==============================================================================
# Cell 5: Execution Loop (Robust Mode)
# ==============================================================================
print("🚀 Starting Big Data Collection Phase...")
print("   This will take time (approx. 1-2 minutes per artist).")
print("   Grab a coffee! ☕")

start_time = time.time()
successful_crawls = 0

for i, artist_name in enumerate(SEED_ARTISTS):
    print(f"\n[{i+1}/{len(SEED_ARTISTS)}] Processing: {artist_name}")
    
    try:
        result = crawl_artist_data(artist_name, SONGS_PER_ARTIST)
        if result:
            successful_crawls += 1
            
        # Polite sleep to keep the API happy
        time.sleep(3)
        
    except Exception as e:
        print(f"⚠️ CRITICAL ERROR crawling {artist_name}: {e}")
        print("   Continuing to next artist...")

total_time = (time.time() - start_time) / 60
print(f"\n✅ Phase 1 Complete.")
print(f"   Successfully crawled {successful_crawls}/{len(SEED_ARTISTS)} artists.")
print(f"   Total time elapsed: {total_time:.2f} minutes.")
print(f"   Check '{RAW_DATA_PATH}' to verify JSON files.")

🚀 Starting Big Data Collection Phase...
   This will take time (approx. 1-2 minutes per artist).
   Grab a coffee! ☕

[1/14] Processing: Kendrick Lamar

--- Starting crawl for: Kendrick Lamar ---
Searching for songs by Kendrick Lamar...

Song 1: "Not Like Us"
Song 2: "HUMBLE."
Song 3: "euphoria"
Song 4: "m.A.A.d city"
Song 5: "Swimming Pools (Drank)"
Song 6: "Money Trees"
Song 7: "DNA."
Song 8: "XXX."
Song 9: "meet the grahams"
Song 10: "Bitch, Don’t Kill My Vibe"
Song 11: "King Kunta"
Song 12: "Poetic Justice"
Song 13: "Alright"
Song 14: "LOVE."
Song 15: "Backseat Freestyle"
Song 16: "Sing About Me, I’m Dying of Thirst"
Song 17: "The Blacker the Berry"
Song 18: "ELEMENT."
Song 19: "LOYALTY."
Song 20: "i"
Song 21: "Wesley’s Theory"
Song 22: "All The Stars"
Song 23: "PRIDE."
Song 24: "FEAR."
Song 25: "6:16 in LA"
Song 26: "A.D.H.D"
Song 27: "Mortal Man"
Song 28: "u"
Song 29: "tv off"
Song 30: "DUCKWORTH."
Song 31: "wacced out murals"
Song 32: "How Much a Dollar Cost"
Song 33: "The Art o

In [13]:
# ==============================================================================
# Cell 6: Data Processing (JSON to CSV)
# ==============================================================================
# Now we turn the raw JSON files into clean tables (CSVs) 
# for Phase 2 (Network) and Phase 3 (NLP).

network_edges = []
song_lyrics = []

print("🔄 Processing raw JSON files into structured CSVs...")

# Iterate through every file in the raw data folder
for filename in os.listdir(RAW_DATA_PATH):
    if filename.endswith(".json"):
        filepath = os.path.join(RAW_DATA_PATH, filename)
        
        # Load the JSON file
        with open(filepath, 'r') as f:
            data = json.load(f)
            
        # The JSON structure from lyricsgenius is: {'name': 'Artist', 'songs': [...]}
        main_artist = data['name']
        songs = data['songs']
        
        print(f"   Processing {len(songs)} songs for {main_artist}...")
        
        for song in songs:
            title = song['title']
            lyrics = song['lyrics']
            release_date = song.get('release_date_for_display', 'N/A')
            
            # --- DATASET 1: LYRICS (For Assignment 3) ---
            # We only keep songs that actually have lyrics
            if lyrics and len(lyrics) > 50:
                song_lyrics.append({
                    'artist': main_artist,
                    'title': title,
                    'year': release_date,
                    'lyrics': lyrics
                })
            
            # --- DATASET 2: NETWORK EDGES (For Assignment 2) ---
            # We look for featured artists to create connections
            if song['featured_artists']:
                for feature in song['featured_artists']:
                    feature_name = feature['name']
                    
                    # Add the edge: Source -> Target
                    network_edges.append({
                        'source': main_artist,
                        'target': feature_name,
                        'song': title
                    })

# Convert lists to Pandas DataFrames
df_edges = pd.DataFrame(network_edges)
df_lyrics = pd.DataFrame(song_lyrics)

# Save to CSV
edges_path = os.path.join(PROCESSED_DATA_PATH, 'network_edges.csv')
lyrics_path = os.path.join(PROCESSED_DATA_PATH, 'song_lyrics.csv')

df_edges.to_csv(edges_path, index=False)
df_lyrics.to_csv(lyrics_path, index=False)

print("\n✅ PROCESSING COMPLETE")
print(f"   Network Edges saved: {len(df_edges)} rows -> {edges_path}")
print(f"   Song Lyrics saved:   {len(df_lyrics)} rows -> {lyrics_path}")

# Show a preview to confirm
print("\n--- Preview: Network Edges ---")
print(df_edges.head())

🔄 Processing raw JSON files into structured CSVs...
   Processing 50 songs for J. Cole...
   Processing 50 songs for The Weeknd...
   Processing 50 songs for Lil Baby...
   Processing 50 songs for Drake...
   Processing 50 songs for Kendrick Lamar...
   Processing 50 songs for Young Thug...
   Processing 50 songs for Tyler, The Creator...
   Processing 50 songs for Nas...
   Processing 50 songs for Eminem...
   Processing 50 songs for Kanye West...
   Processing 50 songs for Future...
   Processing 50 songs for Migos...
   Processing 50 songs for Post Malone...
   Processing 50 songs for Travis Scott...

✅ PROCESSING COMPLETE
   Network Edges saved: 326 rows -> ../data/processed/network_edges.csv
   Song Lyrics saved:   700 rows -> ../data/processed/song_lyrics.csv

--- Preview: Network Edges ---
    source          target              song
0  J. Cole   Amber Coffman         She Knows
1  J. Cole          Miguel        Power Trip
2  J. Cole             TLC     Crooked Smile
3  J. Cole  